In [ ]:
import os

if "notebooks" in os.getcwd():
    %cd ..
from pathlib import Path

import matplotlib.pyplot as plt
import math
import pandas as pd
import numpy as np
import seaborn as sns
from src.utils.defines import INTERIM_DATA_DIR ,AUGMENTED_DATA_DIR, PROCESSED_DATA_DIR, RAW_DATA_DIR
import matplotlib.patches as mpatches

In [389]:
######################################################################################################
# REFORMAT OUTPUT FROM UNLABELED - task a
task_a_unlabeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_a_reddit.csv"))
unlabeled_data = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_data_500k_reddit.csv"))

In [390]:
# create final new data
merged_df = task_a_unlabeled.merge(unlabeled_data, left_index=True, right_index=True, how='inner')

merged_df['target_a'] = np.where(merged_df['0_x'] > merged_df['1'], '0', '1')

merged_df = merged_df.drop(['0_x', '1'], axis=1)
merged_df['text'] = merged_df['0_y']
merged_df = merged_df.drop(['0_y'], axis=1)

merged_df

,target_a,text
0,1,Of course I lose respect for a woman if I fuck...
1,0,Who cares about how many pictures or w.e? If a...
2,0,her twitter name is yikes. lmao
3,0,I agree. It's like giving your kids a vote whe...
4,0,Or you could just have one night stands like t...
...,...,...
499995,0,Like how Stalin made the Ukranians “fear for t...
499996,0,Tbh im not sure of the details behind it. Look...
499997,0,Right... mine was super heavy and I’d often bl...
499998,0,An argument against me saying that the meaning...


In [391]:
merged_df.loc[merged_df['target_a'] == '1']

,target_a,text
0,1,Of course I lose respect for a woman if I fuck...
10,1,If only this foid could actually speak English...
18,1,These are the kinds of posts that give this su...
32,1,She looks hot. probably very high maintenance ...
47,1,Take the condom out of the trash can and rip i...
...,...,...
499941,1,Also saying no to them. And isolated finances ...
499956,1,No one ever said it was OK. Most men are fairl...
499967,1,She needs an excuse to post about it. She is s...
499973,1,Then why not snt the feminazi sexism claim bei...


In [396]:
# take only sexist data
sexist_new = merged_df.loc[merged_df['target_a'] == '1']
# sexist_new.to_csv(f'reddit_task_b.csv', index=False)

sexist_new = sexist_new.drop(['target_a'],axis=1)
# sexist_new.to_csv("unlabeled_task_b_reddit.csv", index=False)
######################################################################################################

In [397]:
# REFORMAT OUTPUT FROM UNLABELED - task b
task_b_labeled = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "probs_task_b_reddit.csv"))
# unlabeled_task_b_GAB = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_task_b_GAB.csv"))
unlabeled_task_b_reddit = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "unlabeled_task_b_reddit.csv"))
# Set a confidence threshold for pseudo-labeling
confidence_threshold = 0.90
test = np.array(task_b_labeled)

# Get the most confident predictions and their indices
most_confident_indices = np.where(test.max(axis=1) > confidence_threshold)[0]
most_confident_preds = pd.DataFrame(test.argmax(axis=1)[most_confident_indices])
most_confident_preds.columns = ['target_b']

text_task_b = unlabeled_task_b_reddit.iloc[most_confident_indices].reset_index(drop=True)

final_task_b = pd.concat([text_task_b, most_confident_preds],axis=1)
final_task_b

,text,target_b
0,Women shouldn’t have rights to falsely accuse ...,3
1,Turn her into a joke. Shame her intelligently....,0
2,Chad (and his buddy Tyrone) probably tag team ...,0
3,I snorted my drink through my nose at the ment...,2
4,Come on.. Even women (FOR FUCK'S SAKE..) are c...,3
...,...,...
6358,"The first allows you to pay women more, menstr...",3
6359,If a woman has a kid knowing the father wants ...,3
6360,Yes women are way more privileged legally. Whi...,3
6361,Also saying no to them. And isolated finances ...,3


In [398]:
unique_values, counts = np.unique(most_confident_preds, return_counts=True)

for value, count in zip(unique_values, counts):
    print(f"{value}: {count}")

0: 830
1: 1594
2: 1793
3: 2146


In [400]:
dummy_values = np.random.rand(len(final_task_b))
df = final_task_b.copy()
df = df.assign(target_a=pd.Series(dummy_values))
df = df.assign(target_c=pd.Series(dummy_values))
df = df.assign(rewire_id=pd.Series(dummy_values))
df = df.reindex(columns=['rewire_id', 'text', 'target_a', 'target_b', 'target_c'])
df.to_csv("task_b_reddit_aug_90conf.csv", index=False)